<a href="https://colab.research.google.com/github/a1869470/Deep_Learning/blob/main/Assignment1_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [252]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
import urllib.request

# Storing the pre-processed data into a variable and retrieving it
data_url = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/diabetes_scale"
urllib.request.urlretrieve(data_url, "diabetes_scale")

# Loading the data using SVMLight format of sklearn as the data is in SVMLight format
X, y = load_svmlight_file("diabetes_scale")

# Pre-processing to convert y to PyTorch tensors (changing class labels from -1 to 0 as pytorch's binary classifcation expects 0 and 1)
y = torch.tensor((y + 1) / 2, dtype=torch.float32)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converting data to PyTorch tensors
X_train = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test = torch.tensor(X_test.toarray(), dtype=torch.float32)

# The Perceptron model with ReLU activation
class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, 1)
        self.relu = nn.ReLU()  # ReLU activation fuction

    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x)  # Apply ReLU activation
        return x

# Create DataLoader objects
batch_size = 128 #Hyperparameter 1
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
model = Perceptron(input_size)
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent and lr as hyperparameter 3

# Train the model
num_epochs = 1000 #Hyperparameter 2
for epoch in range(num_epochs): #Training loop
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    predicted_labels = (test_outputs > 0).float()

accuracy = (predicted_labels == y_test.view(-1, 1)).float().mean().item()
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# # Check if the accuracy is above 70%
# if accuracy > 0.70:
#     print("Accuracy above 70% achieved!")
# else:
#     print("The model did not reach the desired accuracy.")

Accuracy on the test set: 73.38%
